In [1]:
import torch
import numpy as np
import math
import os
import numpy as np
import pandas as pd
import scipy
import matplotlib.pyplot as plt
import seaborn as sns
sns.set()

In [2]:
FILE_PATH = "camera_localization_task5.csv"
col_names = ["timestamp","qr_num","Cx","Cy","w","h","dist","att"]
df = pd.read_csv(FILE_PATH, header=None, names=col_names)


In [3]:
data = pd.DataFrame.to_numpy(df)
s21x = 37
s20x = 25
s25x = 50.5
s26x = 62.5
s27x = 74.5
s31x = 86.5
s32x = 98
sy = 121.5
f = 547
h0 = 11.5

R21 = []
R20 = []
R25 = []
R26 = []
R27 = []
R31 = []
R32 = []

d21 = []
d20 = []
d25 = []
d26 = []
d27 = []
d31 = []
d32 = []


px = torch.tensor([60.], requires_grad=True)
py = torch.tensor([60.], requires_grad=True)
phi = torch.tensor([0.], requires_grad=True)

timestamp = data[0][0]
currentstamp = 0
print(data.shape)
samples = []

(701, 8)


In [4]:
df

,timestamp,qr_num,Cx,Cy,w,h,dist,att
0,1.603874e+09,31,-151,101,79,76,81.578947,-15.646531
1,1.603874e+09,32,-228,101,79,75,82.666667,-22.923659
2,1.603874e+09,21,164,100,81,76,81.578947,16.919423
3,1.603874e+09,27,-72,101,76,77,80.519481,-7.606749
4,1.603874e+09,25,82,100,77,76,81.578947,8.648222
...,...,...,...,...,...,...,...,...
696,1.603874e+09,32,-228,101,80,75,82.666667,-22.923659
697,1.603874e+09,25,82,101,76,76,81.578947,8.648222
698,1.603874e+09,27,-72,102,76,76,81.578947,-7.606749
699,1.603874e+09,26,5,101,74,76,81.578947,0.531357


#underconstruction ignorefornow
def error(sx,sy,px,py,phi, di, angle):
    se = di - torch.sqrt((sx - px)*(sx - px) + (sy - py)*(sy - py))
    pse = angle - (torch.atan((sy - py.detach())/(sx - px.detach()))*(180/math.pi)) + phi
    return se, pse

#underconstruction ignorefornow
for i in range(701):
    currentstamp = data[i][0]
    if(currentstamp == timestamp):
        samples.append(data[i])
    else:
        #fit to model sequentially
        for j in range(len(samples)):
            for epochs in range(4):
                angle = samples[j][7]
                h = samples[j][5]
                di = (f*h0)/h
                if(samples[j][1] == 21):
                    se = di - torch.sqrt((s21x - px)*(s21x - px) + (sy - py)*(sy - py))
                    pse = angle - (torch.atan((sy - py.detach())/(s21x - px.detach()))*(180/math.pi)) + phi
                elif(samples[j][1] == 20):
                    se = di - torch.sqrt((s20x - px)*(s20x - px) + (sy - py)*(sy - py))
                    pse = angle - (torch.atan((sy - py.detach())/(s20x - px.detach()))*(180/math.pi)) + phi
                elif(samples[j][1] == 25):
                    se = di - torch.sqrt((s25x - px)*(s25x - px) + (sy - py)*(sy - py))
                    pse = angle - (torch.atan((sy - py.detach())/(s25x - px.detach()))*(180/math.pi)) + phi
                elif(samples[j][1] == 26):
                    se = di - torch.sqrt((s26x - px)*(s26x - px) + (sy - py)*(sy - py))
                    pse = angle - (torch.atan((sy - py.detach())/(s26x - px.detach()))*(180/math.pi)) + phi
                elif(samples[j][1] == 27):
                    se = di - torch.sqrt((s27x - px)*(s27x - px) + (sy - py)*(sy - py))
                    pse = angle - (torch.atan((sy - py.detach())/(s27x - px.detach()))*(180/math.pi)) + phi
                elif(samples[j][1] == 31):
                    se = di - torch.sqrt((s31x - px)*(s31x - px) + (sy - py)*(sy - py))
                    pse = angle - (torch.atan((sy - py.detach())/(s31x - px.detach()))*(180/math.pi)) + phi
                else:      
                    se = di - torch.sqrt((s32x - px)*(s32x - px) + (sy - py)*(sy - py))
                    pse = angle - (torch.atan((sy - py.detach())/(s32x - px.detach()))*(180/math.pi)) + phi
            
            
                loss = se*se
                loss.backward()
                with torch.no_grad():
                    px -= px.grad * 0.1
                    py -= py.grad * 0.1
                    px.grad.zero_()
                    py.grad.zero_()
            
                losss = pse*pse
                losss.backward()
                with torch.no_grad():
                    phi -= phi.grad * 0.01
                    phi.grad.zero_()
                
    
                
        samples = []
        samples.append(data[i])
        timestamp = currentstamp

       
            
    

In [12]:
di = 83.7725
sx = 62.5
sy = 121.5
angle = 0.5

In [13]:
for i in range(100):
    se = di - torch.sqrt((sx - px)*(sx - px) + (sy - py)*(sy - py))
    see = se*se
    see.backward()
    with torch.no_grad():
        px -= px.grad * 0.1
        py -= py.grad * 0.1
        px.grad.zero_()
        py.grad.zero_()

In [14]:
print(px)
print(py)

tensor([59.0974], requires_grad=True)
tensor([37.7966], requires_grad=True)


In [15]:
for i in range(100):
    pse = angle - torch.atan((sy - py.detach())/(sx - px.detach()))*(180/math.pi) + phi
    loss = pse*pse
    loss.backward()
    with torch.no_grad():
        phi -= phi.grad * 0.1
        phi.grad.zero_()
    

In [16]:
print(phi)

tensor([87.1722], requires_grad=True)
